<p align="center">
  <img src="../img/manfing.png" title="img-principal">
</p>

# CONTEXTO DO PROBLEMA

A **Manfing** é uma empresa que ajuda industrias, varejistas e instituições financeiras a entenderem o comportamento e consumo de seus clientes, para que possam tomar ações mais assertivas. Com isso, os clientes Manfing passam a ter uma previsão de vendas e demanda alinhada a dados de mercado, podendo aumentar seu faturamento e fidelização dos clientes.

A plataforma da Manfing utiliza inteligencia artificial para:

- Reativar clientes.
- Entender o consumo e comportamento.
- Recomendar novos produtos.
- Pricing interno e de concorrentes.
- Encontrar potencial de recompra ou novos SKUs.
- Criar campanhas de WhatsApp para o público alvo.

# O PROBLEMA

Um dos clientes da Manfing possui uma grande quantidade de clientes e entrou em contato, com uma necessidade de entender melhor o comportamento de seus consumidores para direcionar melhor o marketing e entender qual o perfil das pessoas que compram o produto.

# ENTENDIMENTO DO PROBLEMA

**Motivação:**
- Entender o comportamento dos consumidores.

**Causa Raiz do Problema:**
- Diercionar melhor o Marketing.

**Potênciais métodos:**
- Clusterização, Análise de dados.

**Entrega da Solução:**
- Jupyter Notebook.

# PLANEJAMENTO

**Descrição de Dados**
- Entender os dados e verificar nulos.

**Estatistica Descritiva**
- Avaliar estatisticas de primeira ordem e identificar inconsistências.

**Feature Engineering**
- Derivar/criar novas features.

**Análise Exploratória de Dados**
- Realizar análise univariada/bivariada/multivarida para encontrar insights sobre comportamente de clientes.

**Analisar métricas financeiras**
- Realizar análise para encontrar insights sobre consumo financeiro dos clientes.

# 0.0 IMPORTS

In [26]:
# manipulation
import pandas as pd
import inflection
import re

## 0.1 Carregamento de Dados

In [68]:
# lendo dados
data = pd.read_excel('../data/DataSet_Test.xlsx')

# mostrando dados
data.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# 1.0 DESCRIÇÃO DE DADOS

## 1.1 Dimensão de Dados

In [69]:
rows = data.shape[0] # quantidade de linhas
col  = data.shape[1] # quantidade de colunas

# mostrando qtd. linhas e colunas
print('Qty. Columns:', col)
print('Qty. Rows:', rows)

Qty. Columns: 7
Qty. Rows: 522064


## 1.2 Renomeação de Colunas

In [70]:
# função para passar o nome das colunas para o padrão snakecase
snakecase = lambda x: inflection.underscore(x)

# colunas originais
old_columns = data.columns 

# passando para snakecase e substituindo nos dados originais
data.columns = list(map(snakecase, old_columns))

# mostrando novas colunas atuais
data.columns

Index(['bill_no', 'itemname', 'quantity', 'date', 'price', 'customer_id',
       'country'],
      dtype='object')

## 1.3 Tipo de Dados

In [71]:
# verificando se os tipos de dados estão corretos
data.dtypes

bill_no                object
itemname               object
quantity                int64
date           datetime64[ns]
price                 float64
customer_id           float64
country                object
dtype: object

## 1.4 Mudança Tipo de Dados

In [74]:
# bill_no: de str para int
data['bill_no'] = data['bill_no'].astype('int')

# customer_id: float para int
#data['customer_id'] = data['customer_id'].astype('int')

ValueError: invalid literal for int() with base 10: 'A563185'

**ANÁLISE:**

1. Há 3 códigos de compra A563185, A563186, A563187 com uma letra "A" no código, indicando ajuste de debito no item comprado.

**AÇÃO:**

PENDENTE

## 1.4 Dados Faltantes

In [73]:
# verificando se há dados nulos/faltantes
data.isna().sum()

bill_no             0
itemname         1455
quantity            0
date                0
price               0
customer_id    134041
country             0
dtype: int64

## 1.5 Tratamento de Dados Faltantes

In [64]:
data.dropna(inplace=True)

In [67]:
data[data['price'] == 0.0].isna().sum()

bill_no        0
itemname       0
quantity       0
date           0
price          0
customer_id    0
country        0
dtype: int64

# 2.0 ESTATISTICA DESCRITIVA

## 2.1 Variaveis Numéricas

In [52]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,522064.0,10.090435,161.110525,-9600.00,1.00,3.00,10.00,80995.00
price,522064.0,3.826801,41.900599,-11062.06,1.25,2.08,4.13,13541.33
customer_id,388023.0,15316.931710,1721.846964,12346.00,13950.00,15265.00,16837.00,18287.00


**ANÁLISE:** 

1. Há quantidade de produtos negativos (Pode ser devolução)
2. Há preço de produtos negativos (Pode ser devolução)

**AÇÃO:**

PENDENTE

## 2.2 Variaveis Categóricas

In [90]:
categorical = data.select_dtypes(include=['object'])

categorical.apply(lambda x: len(x.unique()))

bill_no     21663
itemname     4186
country        30
dtype: int64

**INSIGHT:** 

- A empresa obteve 21.663 compras unicas.
- A empresa tem 4.186 itens unicos.
- A empresa tem clientes de 30 países distintos.

## 2.3 Variaveis de Data

In [92]:
start = data['date'].min()
end   = data['date'].max()

print('Start Date:', start)
print('End Date  :', end)

Start Date: 2010-12-01 08:26:00
End Date  : 2011-12-09 12:50:00


**INSIGHT:** Há dados de 1 ano da empresa, começando em dezembro de 2010 e termiando em dezembro de 2011.

# 3.0 Feature Engineering

## 3.1 Levantamento de Hipóteses

H1. Os clientes que fazem mais compras ficam no Reino Unido.

H2. Os clientes que gastam mais ficam na Espanha.

H3. Os clientes que levam a maior quantidade de produtos no carrinho ficam no Jpaão.

## 3.2 Criação de Feature

In [88]:
# derivando feature de Ano
data['year'] = data['date'].dt.year

# derivando feature de Mês
data['month'] = data['date'].dt.month

# derivando feature de Dia
data['day'] = data['date'].dt.day

# 4.0 Análise Exploratória de Dados

## 4.1 Análise Univariada

## 4.2 Análise Bivariada

## 4.3 Análise Multivariada